## Multiplication Matrix

The code are explained in the project report

In [2]:
def create_mat_mult(G,EG):
    K = G[0].parent()
    base = K.base()
    x = [K(xi) for xi in K.variable_names()]
    degI = len(EG)
    N = dict()
    n = len(x)
    
    list_EG_values = list(EG.values())
    M = [[[base(0)]*degI]*degI] * n
    M = [Matrix(m) for m in M]
    
    for i,key in enumerate(EG):
        ei = EG[key]
        N[ei] = ei
        for k in range(n):
            if (ei % x[k] == 0):
                ej = ei // x[k]
                if (ej in list_EG_values):
                    j = list_EG_values.index(ej)
                    M[k][i,j] = 1
             
    F = [x[j]*EG[i] for j in range(n) for i in EG if (x[j] * EG[i] not in list_EG_values)]
    F = list(set(F))
    F.sort()
    for t in F:
        for g in G:
            if (t % g.lt() == 0):
                if  (t != g.lt()):
                    for j in range(n):
                        if (t%x[j] == 0):
                            t_prime = t//x[j]
                            if (t_prime in N):
                                if(t_prime not in EG.keys()):
                                    col = parent(M[0][:,0])([0]*degI)
                                    for m,r in list(N[t_prime]):
                                        ind = list_EG_values.index(r)
                                        col += m*M[j][:,ind]
                                    N[t] = 0
                                    for i,j in enumerate(EG):
                                        N[t]+=col[i][0]*EG[j]
                                    for k in range(n):
                                        if (t%x[k] == 0 and t//x[k] in list_EG_values):
                                            i = list_EG_values.index(t//x[k])
                                            M[k][:,i] = col
                else:
                    N[t] = t - g
                    dic_t = dict([(i,j) for j,i in list(N[t])])
                    for k in range(n):
                        if (t % x[k] == 0):
                            ej = t // x[k]
                            if (ej in list_EG_values):
                                j = list_EG_values.index(ej)
                                for i in range(degI):
                                    ei = list_EG_values[i]
                                    if (ei in dic_t):
                                        M[k][i,j] =  dic_t[ei]
    return M,N

## FGLM 

In [3]:
def Update_transition_mat(s,lamb,P_ori):
    P = copy(P_ori)
    degI = P.dimensions()[0]
    for j in lamb.nonzero_positions():
        if j > s:
            k = j
            break
            
    for j in range(degI):
        alpha = P[j,k]/P[k,k]
        P[j,k] = P[s+1,j]
        P[s+1,j] = alpha
        if alpha != 0:
            for i in range(degI):
                if (i != s+1):
                    P[i,j] -= alpha*lamb[i]/lamb[k]
                else:
                    if (lamb[i] != 0):
                        P[i,j] = 1/lamb[i]
    return P 

In [4]:
def FGLM(K,M,EG,N):
    
    def NF(s):
        d = dict([(i,j) for j,i in list(N[s])])
        v = []
        for i,j in enumerate(EG):
            if (j in d.keys()):
                v += [d[j]]
            else:
                v += [0]
        return vector(v)

    x = [K(xi) for xi in K.variable_names()]
    n = len(x)
    S = [K(1)]
    V = [NF(1)]
    L = [(i,0) for i in range(1,n)]
    G = []
    t = (0,0)
    M_space = parent(M[0])
    P = M_space.identity_matrix()
    while(True):
        s = len(S)-1
        k,l = t
        v = M[k]*V[l]
        lamb = P*v
        if lamb[s+1:].is_zero():
            g = x[k]*S[l]-sum([lamb[i]*S[i] for i in range(s+1)])
            if (g != 0):
                G += [g]
        else:
            P = Update_transition_mat(s,lamb,P)
            S += [x[k]*S[l]]
            V += [v]
            L += [(i,s+1) for i in range(n)]
            L = list(set(L))
        L.sort()
        L = [(k,l) for k,l in L if all([x[k]*S[l]%g.lm() != 0 for g in G])]
        if L == []:
            return G
        t = L[0]
        L.remove(t)


In [5]:
def staircase_grobner(G):
    ltG = [g.lt() for g in G]
    K = parent(G[0])
    x = [K(xi) for xi in K.variable_names()]
    n = len(x)
    EG = dict()
    maxdeg = max([g.degree() for g in ltG])
   

    E = []
    for i in range(ZZ([0]*(n-1) + [maxdeg],maxdeg+1)+1):
        l = ZZ(i).digits(maxdeg+1)
        if (sum(l) <= maxdeg*maxdeg):
            E += [l]
    for e in E:
        r = len(e)
        xa = 1
        xa = prod([x[i]^e[i] for i in range(r)])
        if any([xa%g == 0 for g in ltG]):
            continue
        else:
            EG[K(xa)] = K(xa)
    return EG

# Example

In [6]:
x = ['x1','x2','x3','x4']

In [7]:
Q = PolynomialRing(GF(2),names = x,order='lex')
x = [Q(xi) for xi in x] 
x1,x2,x3,x4 = x

In [8]:
G = [Q.random_element(10) for i in range(4)]
G += [x[i]^2 - x[i] for i in range(len(x))]

In [9]:
K = PolynomialRing(GF(2),names = x, order = 'degrevlex')

In [10]:
def is_zero_dim_with_grob_basis(G):
    K = G[0].parent()
    x = [K(xi) for xi in K.variable_names()]
    for xi in x:
        xi_power_in_G = False
        for g in G:
            if (g%xi == 0):
                xi_power_in_G = True
                continue
        if (not(xi_power_in_G)):
            return False
    return True

In [11]:
I = Ideal(G)
G = I.groebner_basis()
is_zero_dim_with_grob_basis(G)

True

In [12]:
EG = staircase_grobner(G)
EG

{1: 1,
 x1: x1,
 x2: x2,
 x1*x2: x1*x2,
 x3: x3,
 x1*x3: x1*x3,
 x2*x3: x2*x3,
 x1*x2*x3: x1*x2*x3,
 x4: x4,
 x2*x4: x2*x4}

In [13]:
M,N = create_mat_mult(G,EG)
M,N

([
[0 0 0 0 0 0 0 0 0 0]  [0 0 0 0 0 0 0 0 0 0]  [0 0 0 0 0 0 0 0 0 0]
[1 1 0 0 0 0 0 0 0 0]  [0 0 0 0 0 0 0 0 0 0]  [0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]  [1 0 1 0 0 0 0 0 0 0]  [0 0 0 0 0 0 0 0 0 0]
[0 0 1 1 0 0 0 0 0 0]  [0 1 0 1 0 0 0 0 0 0]  [0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]  [0 0 0 0 0 0 0 0 0 0]  [1 0 0 0 1 0 0 0 0 0]
[0 0 0 0 1 1 0 0 0 0]  [0 0 0 0 0 0 0 0 0 0]  [0 1 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]  [0 0 0 0 1 0 1 0 0 0]  [0 0 1 0 0 0 1 0 0 0]
[0 0 0 0 0 0 1 1 0 0]  [0 0 0 0 0 1 0 1 0 0]  [0 0 0 1 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0]  [0 0 0 0 0 0 0 0 0 0]  [0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0], [0 0 0 0 0 0 0 0 1 1], [0 0 0 0 0 0 0 0 0 0],

[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 1 0]
[0 0 1 0 0 0 0 0 0 1]
],
 {1: 1,
  x1: x1,
  x2: x2,
  x1*x2: x1*x2,
  x3: x3,
  x1*x3: x1*x3,
  x2*x3: x2*x3,
  x1*x

In [14]:
FGLM(K,M,EG,N)

[x1^2 + x1, x2^2 + x2, x3^2 + x3, x1*x4, x3*x4, x4^2 + x4]

Verify the result with the computation in SageMath

In [15]:
G = [K(r) for r in G]

In [16]:
I = Ideal(G)

In [17]:
I.groebner_basis()

[x1^2 + x1, x2^2 + x2, x3^2 + x3, x1*x4, x3*x4, x4^2 + x4]